# 展开二叉树为链表

### 思路 

本题的思路是原地交换 核心是内部函数顺序 先通过递归得到左边底层节点和右边底层节点 然后交换顺序是先保存 左接右 左删除 右底层节点接保存后的节点 最后返回的时候判断是返回右底层 左底层还是本身即可

### 代码

In [5]:
from collections import deque
from typing import Optional, List

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class SolutionDFS:
    def flatten(self, root: Optional[TreeNode]) -> None:
        def dfs(node):
            if not node:
                return None
            # 递归拉平左右子树
            left_last = dfs(node.left)
            right_last = dfs(node.right)

            # 左子树搬家到右边
            if node.left:
                node_right = node.right
                node.right = node.left
                node.left = None
                left_last.right = node_right

            # 返回当前链表的最后一个节点
            if right_last:
                return right_last
            elif left_last:
                return left_last
            else:
                return node

        dfs(root)

class SolutionBFS:
    def flatten(self, root: Optional[TreeNode]) -> None:
        if not root:
            return
        q = deque([root])
        prev = None
        while q:
            node = q.popleft()
            if prev:
                prev.left = None
                prev.right = node
            # ⚠️ 注意入队顺序：先右后左 → 保证前序顺序
            if node.right:
                q.append(node.right)
            if node.left:
                q.append(node.left)
            prev = node

def levelOrder(root):
    if not root:
        return []
    res, q = [], deque([root])
    while q:
        node = q.popleft()
        if node:
            res.append(node.val)
            q.append(node.left)
            q.append(node.right)
        else:
            res.append(None)
    # 去除末尾多余的 None
    while res and res[-1] is None:
        res.pop()
    return res

def build_demo_tree():
    root = TreeNode(1)
    root.left = TreeNode(2)
    root.right = TreeNode(5)
    root.left.left = TreeNode(3)
    root.left.right = TreeNode(4)
    root.right.right = TreeNode(6)
    return root


root1 = build_demo_tree()
root2 = build_demo_tree()

print("🪴 原始树层序:")
print(levelOrder(root1))

# DFS flatten
SolutionDFS().flatten(root1)
print("\n✅ DFS flatten 后:")
print(levelOrder(root1))

# BFS flatten
SolutionBFS().flatten(root2)
print("\n✅ BFS flatten 后:")
print(levelOrder(root2))


🪴 原始树层序:
[1, 2, 5, 3, 4, None, 6]

✅ DFS flatten 后:
[1, None, 2, None, 3, None, 4, None, 5, None, 6]

✅ BFS flatten 后:
[1, None, 5, None, 2, None, 6, None, 4, None, 3]


### 类似题目（116. 链表连接二叉树）

### 思路

本题的思路BFS比较优异 因此优先考虑 核心就是层序遍历的时候 如果该节点有上一个节点 那么就把上一个节点连上自己 这非常简单 而DFS则需要额外判断自身的子节点的右边有没有节点需要连接 这个判断是通过判断自身有没有下一个节点来决定的 

### 代码

In [13]:
from collections import deque
from typing import Optional

class Node:
    def __init__(self, val: int = 0, left: 'Optional[Node]' = None,
                 right: 'Optional[Node]' = None, next: 'Optional[Node]' = None):
        self.val = val
        self.left = left
        self.right = right
        self.next = next


class SolutionBFS:
    def connect(self, root: 'Optional[Node]') -> 'Optional[Node]':
        if not root:
            return None
        q = deque([root])
        while q:
            size = len(q)
            prev = None
            for _ in range(size):
                node = q.popleft()
                if prev:
                    prev.next = node
                prev = node
                if node.left:
                    q.append(node.left)
                if node.right:
                    q.append(node.right)
        return root


class SolutionDFS:
    def connect(self, root: 'Optional[Node]') -> 'Optional[Node]':
        if not root:
            return None

        def dfs(node):
            if not node or not node.left:
                return
            node.left.next = node.right
            if node.next:
                node.right.next = node.next.left
            dfs(node.left)
            dfs(node.right)

        dfs(root)
        return root

def print_levels_with_next(root: 'Optional[Node]'):
    if not root:
        return
    q = deque([root])
    level = 0
    while q:
        size = len(q)
        nodes = []
        for _ in range(size):
            node = q.popleft()
            nxt = node.next.val if node.next else None
            nodes.append(f"{node.val}->{nxt}")
            if node.left:
                q.append(node.left)
            if node.right:
                q.append(node.right)
        print(f"Level {level}: ", " | ".join(nodes))
        level += 1

def build_demo_tree():
    n1 = Node(1)
    n2 = Node(2)
    n3 = Node(3)
    n4 = Node(4)
    n5 = Node(5)
    n6 = Node(6)
    n7 = Node(7)
    n1.left, n1.right = n2, n3
    n2.left, n2.right = n4, n5
    n3.left, n3.right = n6, n7
    return n1

root_bfs = build_demo_tree()
root_dfs = build_demo_tree()

print("🪴 原始树（无 next）:")
print_levels_with_next(root_bfs)

SolutionBFS().connect(root_bfs)
print("\n✅ BFS 连接结果:")
print_levels_with_next(root_bfs)

SolutionDFS().connect(root_dfs)
print("\n✅ DFS 连接结果:")
print_levels_with_next(root_dfs)


🪴 原始树（无 next）:
Level 0:  1->None
Level 1:  2->None | 3->None
Level 2:  4->None | 5->None | 6->None | 7->None

✅ BFS 连接结果:
Level 0:  1->None
Level 1:  2->3 | 3->None
Level 2:  4->5 | 5->6 | 6->7 | 7->None

✅ DFS 连接结果:
Level 0:  1->None
Level 1:  2->3 | 3->None
Level 2:  4->5 | 5->6 | 6->7 | 7->None
